In [ ]:
!pip install transformers datasets

In [ ]:
import pandas as pd
from datasets import DatasetDict, Dataset
from transformers import TFAutoModel, AutoTokenizer
import tensorflow as tf
import numpy as np

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Projetos/NLP/Sinapses_Classificador/dataset/dataset_gold.csv", sep='#', header=None)

In [ ]:
df = df.rename(columns={0: 'descrição', 1: "label_text", 2: "text", 3: "Tipo"})

In [ ]:
df

,descrição,label_text,text,Tipo
0,Despacho,Despacho,Relação nº 5/2023Fase de LicenciamentoTorna se...,Tipo_Int
1,DESPACHOS DE 11 DE JANEIRO DE 2023,DESPACHO,Assunto: Indeferimento do pedido.Processo Natu...,NaN
2,Despacho,Despacho,Relação nº 14/2023Fase de Autorização de Pesqu...,NaN
3,DESPACHOS DE 9 de janeiro de 2023,DESPACHO,Nº 47/2023 - Ato de Concentração nº 08700.0097...,NaN
4,DESPACHO,DESPACHO,Relação nº 3/2023Fase de Requerimento de Pesqu...,NaN
...,...,...,...,...
2513,"RESOLUÇÃO-RE Nº 304, DE 26 DE JANEIRO DE 2023",RESOLUÇÃO-RE,O Gerente-Geral substituto de Tecnologia de Pr...,2
2514,"RESOLUÇÃO-RE Nº 140, DE 13 DE JANEIRO DE 2023",RESOLUÇÃO-RE,A Gerente-Geral de Inspeção e Fiscalização San...,2
2515,"RESOLUÇÃO-RE Nº 77, DE 12 DE JANEIRO DE 2023",RESOLUÇÃO-RE,O Gerente-Geral substituto de Produtos Biológi...,2
2516,"RESOLUÇÃO-RE Nº 105, DE 12 DE JANEIRO DE 2023",RESOLUÇÃO-RE,A Gerente-Geral de Toxicologia substituta no u...,2


In [ ]:
# Function to map Tipo values to integers (case-insensitive)
def map_tipo_to_int(tipo):
    tipo_lower = tipo.lower()
    if 'despacho' in tipo_lower:
        return 1
    elif 'portaria' in tipo_lower:
        return 2
    elif 'resolução-re' in tipo_lower:
        return 3
    else:
        return tipo

# Apply the mapping function to the 'Tipo' column and create a new 'Mapped_Tipo' column
df['label'] = df["label_text"].apply(map_tipo_to_int)

In [ ]:
df = df[["text","label","label_text"]]

In [ ]:
# Convert values in the 'text' and 'label_text' columns to lowercase
df['label_text'] = df['label_text'].str.lower()
df['text'] = df['text'].str.lower()

<ipython-input-8-d716f061b0da>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label_text'] = df['label_text'].str.lower()
<ipython-input-8-d716f061b0da>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['text'].str.lower()


In [ ]:
df

,text,label,label_text
0,relação nº 5/2023fase de licenciamentotorna se...,1,despacho
1,assunto: indeferimento do pedido.processo natu...,1,despacho
2,relação nº 14/2023fase de autorização de pesqu...,1,despacho
3,nº 47/2023 - ato de concentração nº 08700.0097...,1,despacho
4,relação nº 3/2023fase de requerimento de pesqu...,1,despacho
...,...,...,...
2513,o gerente-geral substituto de tecnologia de pr...,3,resolução-re
2514,a gerente-geral de inspeção e fiscalização san...,3,resolução-re
2515,o gerente-geral substituto de produtos biológi...,3,resolução-re
2516,a gerente-geral de toxicologia substituta no u...,3,resolução-re


In [ ]:
df = df.dropna(subset=['text'])

In [ ]:
df

,text,label,label_text
0,relação nº 5/2023fase de licenciamentotorna se...,1,despacho
1,assunto: indeferimento do pedido.processo natu...,1,despacho
2,relação nº 14/2023fase de autorização de pesqu...,1,despacho
3,nº 47/2023 - ato de concentração nº 08700.0097...,1,despacho
4,relação nº 3/2023fase de requerimento de pesqu...,1,despacho
...,...,...,...
2513,o gerente-geral substituto de tecnologia de pr...,3,resolução-re
2514,a gerente-geral de inspeção e fiscalização san...,3,resolução-re
2515,o gerente-geral substituto de produtos biológi...,3,resolução-re
2516,a gerente-geral de toxicologia substituta no u...,3,resolução-re


In [ ]:
# Define a function to truncate text to a maximum of 512 characters
def truncate_text(text, max_length=512):
    return text[:max_length]

# Apply the truncate_text function to the 'text_column' and store the result in a new column
df['truncated_text'] = df['text'].apply(lambda x: truncate_text(x, max_length=128))

# Print the DataFrame with the truncated text
print(df)

                                                   text  label    label_text  \
0     relação nº 5/2023fase de licenciamentotorna se...      1      despacho   
1     assunto: indeferimento do pedido.processo natu...      1      despacho   
2     relação nº 14/2023fase de autorização de pesqu...      1      despacho   
3     nº 47/2023 - ato de concentração nº 08700.0097...      1      despacho   
4     relação nº 3/2023fase de requerimento de pesqu...      1      despacho   
...                                                 ...    ...           ...   
2513  o gerente-geral substituto de tecnologia de pr...      3  resolução-re   
2514  a gerente-geral de inspeção e fiscalização san...      3  resolução-re   
2515  o gerente-geral substituto de produtos biológi...      3  resolução-re   
2516  a gerente-geral de toxicologia substituta no u...      3  resolução-re   
2517  a coordenadora substituta de autorização de fu...      3  resolução-re   

                                       

<ipython-input-12-079f6ed032ff>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['truncated_text'] = df['text'].apply(lambda x: truncate_text(x, max_length=128))


In [ ]:
df

,text,label,label_text,truncated_text
0,relação nº 5/2023fase de licenciamentotorna se...,1,despacho,relação nº 5/2023fase de licenciamentotorna se...
1,assunto: indeferimento do pedido.processo natu...,1,despacho,assunto: indeferimento do pedido.processo natu...
2,relação nº 14/2023fase de autorização de pesqu...,1,despacho,relação nº 14/2023fase de autorização de pesqu...
3,nº 47/2023 - ato de concentração nº 08700.0097...,1,despacho,nº 47/2023 - ato de concentração nº 08700.0097...
4,relação nº 3/2023fase de requerimento de pesqu...,1,despacho,relação nº 3/2023fase de requerimento de pesqu...
...,...,...,...,...
2513,o gerente-geral substituto de tecnologia de pr...,3,resolução-re,o gerente-geral substituto de tecnologia de pr...
2514,a gerente-geral de inspeção e fiscalização san...,3,resolução-re,a gerente-geral de inspeção e fiscalização san...
2515,o gerente-geral substituto de produtos biológi...,3,resolução-re,o gerente-geral substituto de produtos biológi...
2516,a gerente-geral de toxicologia substituta no u...,3,resolução-re,a gerente-geral de toxicologia substituta no u...


In [ ]:
df = df[["label","label_text","truncated_text"]]

In [ ]:
df = df.rename(columns={'truncated_text': 'text'})
df

,label,label_text,text
0,1,despacho,relação nº 5/2023fase de licenciamentotorna se...
1,1,despacho,assunto: indeferimento do pedido.processo natu...
2,1,despacho,relação nº 14/2023fase de autorização de pesqu...
3,1,despacho,nº 47/2023 - ato de concentração nº 08700.0097...
4,1,despacho,relação nº 3/2023fase de requerimento de pesqu...
...,...,...,...
2513,3,resolução-re,o gerente-geral substituto de tecnologia de pr...
2514,3,resolução-re,a gerente-geral de inspeção e fiscalização san...
2515,3,resolução-re,o gerente-geral substituto de produtos biológi...
2516,3,resolução-re,a gerente-geral de toxicologia substituta no u...


In [ ]:
# Convert DataFrames to Dataset objects
df_dataset = Dataset.from_pandas(df)

In [ ]:
# Remove the '__index_level_0__' column
df_dataset = df_dataset.remove_columns('__index_level_0__')

In [ ]:
df_dataset

Dataset({
    features: ['label', 'label_text', 'text'],
    num_rows: 2517
})

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# Split the DataFrame into training and test sets (e.g., 80% train, 20% test)
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
# Create Dataset for training data
train_dataset = Dataset.from_pandas(train_df)

# Create Dataset for testing data
test_dataset = Dataset.from_pandas(test_df)

In [ ]:
train_dataset

Dataset({
    features: ['label', 'label_text', 'text', '__index_level_0__'],
    num_rows: 2013
})

In [ ]:
# Remove the '__index_level_0__' column
train_dataset = train_dataset.remove_columns('__index_level_0__')

In [ ]:
train_dataset

Dataset({
    features: ['label', 'label_text', 'text'],
    num_rows: 2013
})

In [ ]:
test_dataset

Dataset({
    features: ['label', 'label_text', 'text', '__index_level_0__'],
    num_rows: 504
})

In [ ]:
# Remove the '__index_level_0__' column
test_dataset = test_dataset.remove_columns('__index_level_0__')

In [ ]:
test_dataset

Dataset({
    features: ['label', 'label_text', 'text'],
    num_rows: 504
})

In [ ]:
dataset_dict = DatasetDict({
    'train': train_dataset,
    'test': test_dataset,
})

In [ ]:
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['label', 'label_text', 'text'],
        num_rows: 2013
    })
    test: Dataset({
        features: ['label', 'label_text', 'text'],
        num_rows: 504
    })
})

In [ ]:
model = TFAutoModel.from_pretrained("neuralmind/bert-base-portuguese-cased")

Some layers from the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing TFBertModel: ['mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFBertModel were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['bert/pooler/dense/kernel:0', 'bert/pooler/dense/bias:0']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")

In [ ]:
inputs = tokenizer(['Teste do código', 'Outro teste do CÓDIGO do algoritmo'], padding=True, truncation=True, return_tensors='tf')
inputs

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'input_ids': <tf.Tensor: shape=(2, 11), dtype=int32, numpy=
array([[  101, 15019,   171,  5552,   102,     0,     0,     0,     0,
            0,     0],
       [  101,  5394,  3515,   171,   187, 22369, 18822, 17807,   171,
        11573,   102]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(2, 11), dtype=int32, numpy=
array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(2, 11), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], dtype=int32)>}

In [ ]:
output = model(inputs)
output

TFBaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=<tf.Tensor: shape=(2, 11, 768), dtype=float32, numpy=
array([[[ 0.09256645, -0.3878634 ,  0.24324583, ...,  0.12770069,
         -0.05480366, -0.07042269],
        [ 0.21420066, -0.15056115,  0.5004485 , ...,  0.00888378,
         -0.34603256, -0.65474266],
        [ 0.3029013 , -0.14671797,  0.52353805, ..., -0.21340795,
         -0.50538766, -0.16456725],
        ...,
        [ 0.38791376, -0.11334046,  0.5060785 , ...,  0.03669767,
         -0.44881463, -0.5225777 ],
        [ 0.03812381, -0.26503533,  0.23004484, ..., -0.1860592 ,
         -0.3024553 ,  0.27946067],
        [ 0.54386604, -0.21479201,  0.5647856 , ...,  0.11791121,
         -0.3689448 , -0.13068879]],

       [[ 0.39921626, -0.27855647,  0.32920238, ..., -0.16217309,
          0.3201584 , -0.02952561],
        [ 0.36867225, -0.24360706,  0.48498502, ..., -0.25687882,
         -0.27961996, -0.15797386],
        [ 0.18037221, -0.15503253,  0.44241583, ..

In [ ]:
def tokenize(batch):
  return tokenizer(batch["text"], padding=True, truncation=True)

In [ ]:
dataset_encoded = dataset_dict.map(tokenize, batched=True, batch_size=None)

Map:   0%|          | 0/2013 [00:00<?, ? examples/s]

Map:   0%|          | 0/504 [00:00<?, ? examples/s]

In [ ]:
dataset_encoded

DatasetDict({
    train: Dataset({
        features: ['label', 'label_text', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2013
    })
    test: Dataset({
        features: ['label', 'label_text', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 504
    })
})

In [ ]:
dataset_encoded.set_format('tf', columns=['input_ids', 'attention_mask', 'token_type_ids', 'label'])

In [ ]:
policy = tf.keras.mixed_precision.Policy('mixed_float16')
tf.keras.mixed_precision.set_global_policy(policy)

In [ ]:
BATCH_SIZE = 64

def order(inp):
  data = list(inp.values())
  return {
      'input_ids':data[1],
      'attention_mask':data[2],
      'token_type_ids':data[3]
  }, data[0]

# converting train split of 'emotions_encoded' to tensorflow format
train_dataset = tf.data.Dataset.from_tensor_slices(dataset_encoded['train'][:])
# set batch_size and shuffle
train_dataset = train_dataset.batch(BATCH_SIZE).shuffle(1000)
# map the 'order' function
train_dataset = train_dataset.map(order, num_parallel_calls=tf.data.AUTOTUNE)

# .. doing the same for test set ...
test_dataset = tf.data.Dataset.from_tensor_slices(dataset_encoded['test'][:])
test_dataset = test_dataset.batch(BATCH_SIZE)
test_dataset = test_dataset.map(order, num_parallel_calls=tf.data.AUTOTUNE)

In [ ]:
inp, out = next(iter(train_dataset))
print(inp, '\n\n', out)

{'input_ids': <tf.Tensor: shape=(64, 63), dtype=int64, numpy=
array([[  101,   146,  1640, ...,     0,     0,     0],
       [  101,   123, 18082, ...,     0,     0,     0],
       [  101,  1712,   100, ...,     0,     0,     0],
       ...,
       [  101,   146,  2481, ...,     0,     0,     0],
       [  101,   123,  1229, ...,     0,     0,     0],
       [  101,   146,  1640, ...,     0,     0,     0]])>, 'attention_mask': <tf.Tensor: shape=(64, 63), dtype=int64, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])>, 'token_type_ids': <tf.Tensor: shape=(64, 63), dtype=int64, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])>} 

 tf.Tensor(
[2 2 1 3 2 1 2 1 2 3 1 2 2 

In [ ]:
class BERTForClassification(tf.keras.Model):

  def __init__(self, bert_model, num_classes):
    super().__init__()
    self.bert = bert_model
    self.fc = tf.keras.layers.Dense(num_classes, activation='softmax')

  def call(self, inputs):
    x = self.bert(inputs)[1]
    return self.fc(x)

In [ ]:
classifier = BERTForClassification(model, num_classes=6)

classifier.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

In [ ]:
history = classifier.fit(
    train_dataset,
    epochs=3,
    batch_size=4
)

Epoch 1/3
32/32 [==============================] - 70s 879ms/step - loss: 1.0201 - accuracy: 0.6001
Epoch 2/3
32/32 [==============================] - 23s 726ms/step - loss: 0.4339 - accuracy: 0.8549
Epoch 3/3
32/32 [==============================] - 24s 735ms/step - loss: 0.1469 - accuracy: 0.9588


In [ ]:
classifier.evaluate(test_dataset)

8/8 [==============================] - 5s 218ms/step - loss: 0.0911 - accuracy: 0.9702


[0.0911124125123024, 0.9702380895614624]

In [ ]:
# Define a single input instance (replace this with your actual data)
input_text = "Processo nº 48500.003134/2012-10. Interessado: Migratio Gestão e Comercialização de Energia Elétrica Ltda. Decisão: Autorizar a empresa Migratio Gestão e Comercialização de Energia Elétrica Ltda., inscrita no CNPJ/MF sob nº 15.458.171/0001-36, a atuar como Agente Comercializador de Energia Elétrica no âmbito da CCEE; informar que a atividade poderá ser exercida por meio de sua filial, CNPJ/MF sob nº 15.458.171/0002-17. A íntegra deste Despacho consta dos autos e encontra-se disponível em www.aneel.gov.br/biblioteca."

# Tokenize and preprocess the input instance
# You need to perform the same tokenization and preprocessing as you did for your dataset
# Assuming you have access to a tokenizer that matches your BERT model
input_tokens = tokenizer(input_text, padding=True, truncation=True)  # Replace with your tokenizer instance

In [ ]:
input_tokens

{'input_ids': [101, 22199, 100, 8659, 8418, 119, 14736, 12763, 10476, 120, 3470, 118, 1193, 119, 1508, 2061, 243, 131, 10192, 522, 3783, 15093, 122, 13522, 910, 125, 16837, 1801, 8255, 21017, 119, 4534, 758, 131, 14514, 2854, 123, 1799, 10192, 522, 3783, 15093, 122, 13522, 910, 125, 16837, 1801, 8255, 21017, 119, 117, 5813, 154, 202, 12159, 22310, 22333, 120, 213, 22324, 425, 100, 997, 119, 6723, 22330, 119, 13500, 120, 6324, 22302, 118, 6408, 117, 123, 6795, 271, 22026, 175, 13522, 16184, 125, 16837, 1801, 8255, 202, 6665, 180, 17374, 22309, 22309, 132, 21424, 179, 123, 3546, 8209, 333, 6569, 328, 240, 1423, 125, 327, 21495, 117, 12159, 22310, 22333, 120, 213, 22324, 425, 100, 997, 119, 6723, 22330, 119, 13500, 120, 6324, 22313, 118, 1040, 119, 177, 14204, 22279, 522, 2166, 1305, 321, 2014, 8051, 298, 1846, 22281, 122, 1377, 118, 176, 5656, 173, 2702, 11740, 119, 15631, 178, 119, 3746, 22292, 119, 235, 22282, 120, 7770, 119, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
# Tokenize and preprocess the input text
#input_tokens = tokenizer.encode(input_text, max_length=max_sequence_length, padding='max_length', truncation=True)
input_ids = input_tokens['input_ids']
attention_mask = input_tokens['attention_mask']
token_type_ids = input_tokens['token_type_ids']

# Convert the preprocessed input into a TensorFlow tensor
input_ids = tf.convert_to_tensor([input_ids])
attention_mask = tf.convert_to_tensor([attention_mask])
token_type_ids = tf.convert_to_tensor([token_type_ids])

# Make predictions using the classifier
predictions = classifier.predict([input_ids, attention_mask, token_type_ids])

# Get the predicted class
predicted_class = np.argmax(predictions, axis=-1)[0]  # Assuming you have a single instance

# Print the predicted class
print("Predicted Class:", predicted_class)

1/1 [==============================] - 0s 63ms/step
Predicted Class: 1
